<a href="https://colab.research.google.com/github/ilham-mukti/Machine-Learning/blob/main/data_teks/Document_Similarity_Data_Tirto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
import nltk
nltk.download("stopwords")
nltk.download("punkt")

import numpy as np
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer

sw_indo = stopwords.words("indonesian") + list(punctuation)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [71]:
datas = pd.read_csv("/content/data_tirto.csv")
datas.head()

,id_berita,judul_berita,isi_berita,nama_penulis,date_created,url_artikel
0,1259889,Jokowi Sindir ASN yang Minta Dilayani: Seperti...,Presiden Jokowi menilai para aparatur sipil ne...,Andrian Pratama Taher,27/07/2021 10:55,https://tirto.id/jokowi-sindir-asn-yang-minta-...
1,1259829,Larangan Demo Jokowi End Game & Bahaya Doxing ...,"Jumat, 23 Juli 2021, sekira pukul 23-an, ada t...",Adi Briantika,26/07/2021 17:24,https://tirto.id/larangan-demo-jokowi-end-game...
2,1259921,"Jokowi: Mahasiswa Punya Talenta, Bisa Berprofe...",Presiden Joko Widodo meminta agar perguruan ti...,Andrian Pratama Taher,27/07/2021 13:36,https://tirto.id/jokowi-mahasiswa-punya-talent...
3,1259902,Tjahjo Minta Seluruh ASN Ikuti Arahan Jokowi-M...,Menteri Pendayagunaan Aparatur Negara dan Refo...,Andrian Pratama Taher,27/07/2021 12:30,https://tirto.id/tjahjo-minta-seluruh-asn-ikut...
4,1259792,"Jokowi, Menkes dan Mendag Disomasi Imbas Kelan...",Koalisi masyarakat sipil menyomasi Presiden Jo...,Irwan Syambudi,26/07/2021 13:41,https://tirto.id/jokowi-menkes-dan-mendag-diso...


In [142]:
datas.shape

(1303, 6)

In [143]:
datas.drop_duplicates(subset='id_berita', inplace=True, ignore_index=True)

In [144]:
datas.shape

(863, 6)

# Ekstrak TFIDF

In [145]:
tfidf = TfidfVectorizer(ngram_range=(1,2), tokenizer=word_tokenize, stop_words=sw_indo)
tfidf_matrix = tfidf.fit_transform(datas.isi_berita)
tfidf_matrix

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['``'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


<863x243099 sparse matrix of type '<class 'numpy.float64'>'
	with 475069 stored elements in Compressed Sparse Row format>

# Document Similarity

In [160]:
def document_similarity(index, matrix, topk=10):
  sim = cosine_similarity(matrix[index], matrix)
  sorted_sim = sim[0].argsort()
  berita_yang_sama = [index for index in reversed(sorted_sim[-topk:])]
  return berita_yang_sama

In [163]:
data_ke = 20
result_index = document_similarity(data_ke, tfidf_matrix)

In [164]:
datas.loc[result_index]

,id_berita,judul_berita,isi_berita,nama_penulis,date_created,url_artikel
20,1260055,Eks Mensos Juliari Dituntut 11 Tahun Penjara &...,Mantan Menteri Sosial Juliari Batubara dituntu...,Alfian Putra Abdi,28/07/2021 14:10,https://tirto.id/eks-mensos-juliari-dituntut-1...
17,1260026,Eks Mensos Juliari Jalani Sidang Tuntutan Koru...,Mantan Menteri Sosial Juliari Batubara akan me...,Alfian Putra Abdi,28/07/2021 11:16,https://tirto.id/eks-mensos-juliari-jalani-sid...
501,1259187,KPK Ajukan Banding Atas Vonis Eks Panitera PN ...,Jaksa Penuntut Umum Komisi Pemberantasan Korup...,Mohammad Bernie,19/07/2021 14:15,https://tirto.id/kpk-ajukan-banding-atas-vonis...
512,1258654,ICW Kecewa Gugatan Ganti Rugi Bansos Kasus Jul...,Indonesia Corruption Watch (ICW) kecewa dengan...,Mohammad Bernie,14/07/2021 10:14,https://tirto.id/icw-kecewa-gugatan-ganti-rugi...
505,1258955,"Sikapi Vonis Eks Menteri KKP Eddy Prabowo, KPK...",Komisi Pemberantasan Korupsi (KPK) menunggu sa...,tidak ada,16/07/2021 14:13,https://tirto.id/sikapi-vonis-eks-menteri-kkp-...
510,1258875,Edhy Prabowo Dihukum 5 Tahun Penjara Dalam Kas...,Majelis Hakim Pengadilan Tindak Pidana Korupsi...,Mohammad Bernie,15/07/2021 17:26,https://tirto.id/edhy-prabowo-dihukum-5-tahun-...
509,1258880,"5 Perantara Suap Edhy Prabowo Divonis 4-4,5 Ta...",Pengadilan Tindak Pidana Korupsi (Tipikor) Jak...,Mohammad Bernie,15/07/2021 18:08,https://tirto.id/5-perantara-suap-edhy-prabowo...
530,1258014,Dua WNA Italia Divonis Bebas di Kasus Mafia Ta...,Dua warga negara asing (WNA) Italia divonis be...,tidak ada,07/07/2021 17:26,https://tirto.id/dua-wna-italia-divonis-bebas-...
533,1257901,Penyidik KPK Nonaktif: Kerugian Korupsi Bansos...,Penyidik nonaktif Komisi Pemberantasan Korupsi...,Mohammad Bernie,06/07/2021 17:01,https://tirto.id/penyidik-kpk-nonaktif-kerugia...
515,1258682,Tak Puas Nurhadi hanya Dihukum 6 Tahun Penjara...,Jaksa Komisi Pemberantasan Korupsi (KPK) yang ...,Mohammad Bernie,14/07/2021 12:17,https://tirto.id/tak-puas-nurhadi-hanya-dihuku...


In [165]:
def extract_keyword_tfidf(doc, tfidf, topk=10):
  matrix = tfidf.transform([doc])
  vocab = tfidf.get_feature_names()

  sorted_tfidf = matrix[0].toarray()[0].argsort()
  return([vocab[idx] for idx in reversed(sorted_tfidf[-topk:])])

In [166]:
text = datas.isi_berita[20]
extract_keyword_tfidf(text, tfidf)

['juliari',
 'jpu',
 'bansos covid-19',
 'jo pasal',
 'bansos',
 'jo',
 'pasal',
 "'' jpu",
 'proyek pengadaan',
 'pengadaan bansos']